# Análisis Exploratorio de Datos (EDA) para Mantenimiento Predictivo del Motocompresor C-5080

**Objetivo:** Realizar un análisis exhaustivo de los datos de sensores y del historial de fallas del motocompresor C-5080 para identificar patrones, anomalías y correlaciones que sirvan como base para desarrollar un modelo de mantenimiento predictivo.

**Autor:** [Tu Nombre]
**Fecha:** [Fecha Actual]
**Curso:** Proyecto de Grado

## 0. Importación de Librerías

Importamos las librerías necesarias para el análisis de datos y la visualización.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de estilos para los gráficos
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (16, 8)

## 1. Carga e Inspección Inicial de Datos

Cargamos los archivos CSV proporcionados. Los datos de sensores se consolidan en un único DataFrame y se realiza una primera inspección para entender su estructura y contenido.

In [ ]:
# --- ¡IMPORTANTE! Reemplaza estas rutas con la ubicación real de tus archivos ---
paths_sensores = [
    '01-2024.xls - Compresor #1.csv',
    '04-2025.xls - Compresor #1.csv',
    '8-2023...xls - Compresor #1.csv'
]
path_fallas = 'Historial C1 RGD.xlsx - Table 1.csv'
# --------------------------------------------------------------------------------

try:
    # Cargar y concatenar datos de sensores, omitiendo las primeras filas de metadatos
    lista_df_sensores = [pd.read_csv(p, skiprows=4) for p in paths_sensores]
    df_sensores = pd.concat(lista_df_sensores, ignore_index=True)

    # Cargar historial de fallas
    df_fallas = pd.read_csv(path_fallas, skiprows=3)
    print("✅ Archivos cargados exitosamente.")
except FileNotFoundError as e:
    print(f"❌ Error: Archivo no encontrado. Verifica las rutas. Detalle: {e}")

print("\n--- Inspección del DataFrame de Sensores ---")
df_sensores.info()
print("\nPrimeras filas de datos de Sensores:")
display(df_sensores.head())

print("\n--- Inspección del DataFrame de Fallas ---")
df_fallas.info()
print("\nPrimeras filas de datos de Fallas:")
display(df_fallas.head())

## 2. Limpieza y Preprocesamiento de Datos

La calidad de los datos es fundamental. En este paso, convertimos las columnas de fecha a un formato adecuado, manejamos los valores faltantes y eliminamos inconsistencias.

In [ ]:
# --- Preprocesamiento de df_sensores ---
# Renombrar columnas para facilitar el acceso
df_sensores.columns = [col.replace('\n', ' ').strip() for col in df_sensores.columns]

# Convertir 'Hora' a datetime. El formato numérico de Excel se interpreta como días desde 1899-12-30.
df_sensores['Timestamp'] = pd.to_datetime(df_sensores['Hora'], unit='d', origin='1899-12-30', errors='coerce')
df_sensores = df_sensores.drop(columns=['Hora', 'ESTADO'])
df_sensores = df_sensores.sort_values('Timestamp').reset_index(drop=True)

# Manejo de valores faltantes
print(f"Valores faltantes antes de la limpieza: {df_sensores.isnull().sum().sum()}")
# Usamos interpolación lineal, adecuada para series temporales
df_sensores.interpolate(method='linear', limit_direction='forward', inplace=True)
df_sensores.dropna(inplace=True) # Eliminar filas que no se pudieron interpolar (ej. al inicio)
print(f"Valores faltantes después de la limpieza: {df_sensores.isnull().sum().sum()}")

# --- Preprocesamiento de df_fallas ---
df_fallas.rename(columns={'FECHA INICIO': 'Fecha_Inicio', 'FECHA FIN': 'Fecha_Fin'}, inplace=True)
df_fallas['Fecha_Inicio'] = pd.to_datetime(df_fallas['Fecha_Inicio'], dayfirst=True, errors='coerce')
df_fallas['Fecha_Fin'] = pd.to_datetime(df_fallas['Fecha_Fin'], dayfirst=True, errors='coerce')
df_fallas.dropna(subset=['Fecha_Inicio'], inplace=True)

print("\n✅ Preprocesamiento completado.")

## 3. Análisis Estadístico Descriptivo

Obtenemos un resumen cuantitativo de las variables numéricas para entender sus tendencias centrales y dispersión.

In [ ]:
print("--- Resumen Estadístico de las Variables de Sensores ---")
display(df_sensores.describe().T)

## 4. Análisis Univariado

Visualizamos la distribución de las variables más críticas para identificar su forma, sesgo y la presencia de valores atípicos.

In [ ]:
variables_criticas = [
    'RPM',
    'Presión Descarga',
    'Pres. Aceite Comp',
    'Temp. Cilindro # 1',
    'Temp. Cilindro # 2',
    'Temp. Cilindro # 3',
    'Temp. Cilindro # 4',
    'Presión Aceite Motor',
    'Temp. Aceite Motor',
    'Temp. Agua Motor'
]

for var in variables_criticas:
    if var in df_sensores.columns:
        plt.figure(figsize=(18, 6))
        
        # Histograma y KDE
        plt.subplot(1, 2, 1)
        sns.histplot(df_sensores[var], kde=True, bins=50)
        plt.title(f'Distribución de {var}')
        plt.xlabel(var)
        plt.ylabel('Frecuencia')
        
        # Gráfico de Caja
        plt.subplot(1, 2, 2)
        sns.boxplot(x=df_sensores[var])
        plt.title(f'Gráfico de Caja de {var}')
        plt.xlabel(var)
        
        plt.tight_layout()
        plt.show()
        print(f"Interpretación para {var}: La media es {df_sensores[var].mean():.2f} y la desviación estándar es {df_sensores[var].std():.2f}. El gráfico de caja muestra la presencia de outliers [arriba/abajo/ambos].")
        print("-"*80)

## 5. Análisis Bivariado y Multivariado

Investigamos las relaciones entre pares de variables para descubrir dependencias que puedan ser clave para la predicción de fallas.

In [ ]:
# Matriz de Correlación
correlation_matrix = df_sensores[variables_criticas].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Mapa de Calor de Correlación entre Variables Críticas')
plt.show()

print("Interpretación: El mapa de calor muestra las correlaciones de Pearson. Valores cercanos a 1 (rojo oscuro) indican una fuerte correlación positiva, mientras que valores cercanos a -1 (azul oscuro) indican una fuerte correlación negativa. Valores cercanos a 0 (blanco) sugieren poca o ninguna correlación lineal.")

In [ ]:
# Gráficos de Dispersión para pares altamente correlacionados
print("\n--- Gráficos de Dispersión ---")
sns.scatterplot(data=df_sensores, x='RPM', y='Presión Descarga')
plt.title('RPM vs. Presión de Descarga')
plt.show()
print("Interpretación: Se observa la relación entre las RPM y la presión de descarga. Un patrón claro puede indicar una fuerte dependencia funcional.")

sns.scatterplot(data=df_sensores, x='Temp. Aceite Motor', y='Temp. Agua Motor')
plt.title('Temperatura del Aceite vs. Temperatura del Agua del Motor')
plt.show()
print("Interpretación: Ambas temperaturas deberían estar correlacionadas, ya que el sistema de enfriamiento afecta a ambos fluidos.")

## 6. Análisis de Series Temporales

Visualizamos la evolución de las variables a lo largo del tiempo para detectar tendencias, estacionalidades o cambios abruptos.

In [ ]:
df_sensores_resampled = df_sensores.set_index('Timestamp').resample('D').mean().reset_index()

plt.figure(figsize=(20, 8))
plt.plot(df_sensores_resampled['Timestamp'], df_sensores_resampled['RPM'], label='RPM Promedio Diario')
plt.title('Evolución de las RPM a lo largo del Tiempo (Promedio Diario)')
plt.xlabel('Fecha')
plt.ylabel('RPM')
plt.legend()
plt.show()

plt.figure(figsize=(20, 8))
plt.plot(df_sensores_resampled['Timestamp'], df_sensores_resampled['Temp. Aceite Motor'], label='Temp. Aceite Motor', color='orange')
plt.plot(df_sensores_resampled['Timestamp'], df_sensores_resampled['Temp. Agua Motor'], label='Temp. Agua Motor', color='blue')
plt.title('Evolución de las Temperaturas del Motor (Promedio Diario)')
plt.xlabel('Fecha')
plt.ylabel('Temperatura (°F)')
plt.legend()
plt.show()

## 7. Análisis de Fallas (Integración de Datasets)

Este es el paso más importante. Cruzamos los datos de fallas con los datos de sensores para buscar patrones que puedan predecir un evento adverso.

In [ ]:
df_sensores_ts = df_sensores.set_index('Timestamp')

# Seleccionar una falla específica para analizar
# Ejemplo: Analicemos la primera falla documentada
if not df_fallas.empty:
    falla_ejemplo = df_fallas.iloc[0]
    fecha_falla = falla_ejemplo['Fecha_Inicio']
    descripcion_falla = falla_ejemplo['DESCRIPCION']

    print(f"Analizando la falla: '{descripcion_falla}' ocurrida el {fecha_falla.date()}\n")

    # Definir la ventana de tiempo para el análisis (ej. 48 horas antes de la falla)
    ventana_inicio = fecha_falla - pd.Timedelta(hours=48)
    ventana_fin = fecha_falla

    # Filtrar los datos de sensores en esa ventana
    datos_pre_falla = df_sensores_ts.loc[ventana_inicio:ventana_fin]

    if not datos_pre_falla.empty:
        # Graficar una variable crítica antes de la falla
        var_a_analizar = 'Presión Aceite Motor'
        plt.figure(figsize=(20, 8))
        plt.plot(datos_pre_falla.index, datos_pre_falla[var_a_analizar], marker='o', linestyle='-', label=f'{var_a_analizar} antes de la falla')
        plt.axvline(fecha_falla, color='red', linestyle='--', lw=2, label=f'Momento de la Falla: {descripcion_falla}')
        plt.title(f'Comportamiento de {var_a_analizar} en las 48h previas a la falla')
        plt.xlabel('Fecha y Hora')
        plt.ylabel(var_a_analizar)
        plt.legend()
        plt.show()
        print(f"Interpretación: Este gráfico muestra si la variable '{var_a_analizar}' tuvo un comportamiento anómalo (tendencia, pico, caída) justo antes del evento. Esto podría ser un indicador predictivo.")
    else:
        print(f"No se encontraron datos de sensores en la ventana de 48 horas antes de la falla del {fecha_falla.date()}")
else:
    print("El dataframe de fallas está vacío. No se puede realizar el análisis de fallas.")

## 8. Síntesis y Conclusiones del EDA

Aquí resumimos los hallazgos clave y proponemos los siguientes pasos para el proyecto de grado.

### Hallazgos Principales:
1.  **Calidad de los Datos:** Los datos de sensores requieren una limpieza inicial, especialmente en el manejo de fechas y la interpolación de valores faltantes para crear una serie temporal continua.
2.  **Correlaciones Clave:** Se identificaron fuertes correlaciones entre [Mencionar un par de variables, ej., RPM y Temperatura de Descarga] y [Mencionar otro par]. Estas relaciones son lógicas desde un punto de vista mecánico y serán fundamentales para la ingeniería de características.
3.  **Distribuciones y Outliers:** Variables como [Mencionar una variable] muestran una distribución [normal/sesgada], mientras que [Mencionar otra variable] presenta una cantidad significativa de outliers que deben ser investigados. Podrían representar mediciones erróneas o verdaderos eventos anómalos.
4.  **Patrones Precursores de Fallas:** El análisis de la ventana de tiempo previa a la falla de [Mencionar falla analizada] mostró un comportamiento anómalo en la variable [Mencionar variable analizada], sugiriendo que es un fuerte candidato para ser un indicador predictivo.

### Recomendaciones y Siguientes Pasos:
1.  **Ingeniería de Características (Feature Engineering):**
    * Crear características de ventana deslizante (ej., media móvil, desviación estándar móvil en ventanas de 1, 6 y 12 horas) para capturar tendencias a corto plazo.
    * Generar variables de interacción basadas en las correlaciones encontradas (ej., ratio `Presión Descarga / RPM`).
    * Crear características de retardo (lag features) para usar valores pasados como predictores.
2.  **Etiquetado de Datos para Modelado:**
    * Definir una ventana de tiempo antes de cada falla (ej., 24 horas) y etiquetar todos los puntos de datos dentro de esa ventana como "pre-falla" (1) y el resto como "normal" (0). Esto creará el objetivo para un modelo de clasificación binaria.
3.  **Modelado de Machine Learning:**
    * Comenzar con modelos de clasificación robustos como `Random Forest` o `Gradient Boosting (XGBoost)` para predecir la probabilidad de una falla inminente.
    * Utilizar las métricas de evaluación adecuadas para problemas desbalanceados (Precision, Recall, F1-Score, AUC-ROC) ya que los eventos de falla serán mucho menos frecuentes que la operación normal.